In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=02c04e114564d90d62546c36c6f6d19a30665936696e99ef4408a3fb9f3f24a3
  Stored in directory: /Users/tc/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [17]:
X = df.drop('koi_disposition', axis=1)
X.columns

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk',
       'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1',
       'koi_impact_err2', 'koi_duration', 'koi_duration_err1',
       'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
       'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg',
       'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1',
       'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

In [24]:
from sklearn.feature_selection import SelectKBest, f_classif
# select 10 best features
select_kbest = SelectKBest(f_classif, k=10)
X_sel = select_kbest.fit_transform(X, y)
X_sel.shape

(6991, 10)

In [29]:
mask = select_kbest.get_support()
new_features = []
for m, col in zip(mask, df.columns[1:]):
    if m:
        new_features.append(col)
new_features

['koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_fpflag_co',
 'koi_fpflag_ec',
 'koi_depth',
 'koi_teq',
 'koi_model_snr',
 'koi_steff_err1',
 'koi_steff_err2',
 'koi_slogg_err2']

In [30]:
# Set features. This will also be used as your x values.
selected_features = df[new_features]
selected_features

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_depth,koi_teq,koi_model_snr,koi_steff_err1,koi_steff_err2,koi_slogg_err2
0,0,0,0,0,874.8,443,25.8,81,-81,-0.096
1,0,1,0,0,10829.0,638,76.3,158,-176,-0.176
2,0,1,0,0,8079.2,1395,505.6,157,-174,-0.168
3,0,0,0,0,603.3,1406,40.9,169,-211,-0.210
4,0,0,0,0,686.0,1160,40.2,189,-232,-0.229
...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,87.7,929,8.4,169,-152,-0.189
6987,0,1,1,0,1579.2,2088,453.3,139,-166,-0.196
6988,0,0,0,0,48.5,1608,10.6,165,-220,-0.224
6989,0,0,1,0,103.6,2218,12.3,193,-236,-0.224


# Create a Train Test Split

Use `koi_disposition` for the y values

In [32]:
# encoding the target label
from sklearn.preprocessing import LabelEncoder

y = df['koi_disposition']
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
y

array([1, 2, 2, ..., 0, 2, 2])

In [45]:
# split the dataset to train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2)

In [46]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_depth,koi_teq,koi_model_snr,koi_steff_err1,koi_steff_err2,koi_slogg_err2
6587,1,0,0,0,299.3,316,9.2,166,-166,-0.160
5910,0,1,1,0,153.0,5119,9.8,54,-68,-0.098
4720,1,0,1,0,312.8,363,8.1,196,-236,-0.334
267,0,1,1,1,4399.1,1170,270.1,163,-181,-0.192
3680,0,1,0,0,387900.0,1905,2624.0,69,-88,-0.119


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [47]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled[0]

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.89179614e-04, 1.53687822e-02, 8.39491445e-04, 2.45562130e-01,
       9.04212349e-01, 8.41112214e-01])

# Train the Model



In [ ]:
# random forest classifier
model = 

In [43]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

NameError: name 'model2' is not defined

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)